<a href="https://colab.research.google.com/github/greatpak/Breast_cancer_detection/blob/master/Query_Big_Query_from_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [6]:
# Set your Google Cloud Project ID
project_id = 'sqlcourse-352110'

In [7]:
from google.cloud import bigquery
import pandas as pd

# Create a BigQuery client
client = bigquery.Client(project=project_id)

In [8]:
query = """
          SELECT name, SUM(number) as count
          FROM `bigquery-public-data.usa_names.usa_1910_current`
          GROUP BY name
          ORDER BY count DESC
          LIMIT 10
        """

query_job = client.query(query)

AttributeError: module 'google.cloud.bigquery' has no attribute 'retry'

In [9]:
df = query_job.to_dataframe()
df

NameError: name 'query_job' is not defined

In [ ]:
import os
import json
import psycopg2
from psycopg2.extras import execute_values
from google.cloud import bigquery
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from dotenv import load_dotenv

# ===============================
# 🔹 Load secrets from .env file
# ===============================
load_dotenv()  # will read .env file if present

REDSHIFT_CONFIG = {
    "database": os.getenv("REDSHIFT_DB"),
    "user": os.getenv("REDSHIFT_USER"),
    "password": os.getenv("REDSHIFT_PASSWORD"),
    "host": os.getenv("REDSHIFT_HOST"),
    "port": int(os.getenv("REDSHIFT_PORT", 5439))
}

SSH_CONFIG = {
    "ssh_host": os.getenv("SSH_HOST"),
    "ssh_port": int(os.getenv("SSH_PORT", 22)),
    "ssh_user": os.getenv("SSH_USER"),
    "ssh_password": os.getenv("SSH_PASSWORD"),
}

BQ_PROJECT_ID = os.getenv("BQ_PROJECT_ID", "gtm-mkxz6jtz-ndzln")

BQ_QUERY = """
    SELECT *
    FROM `indigo-codex-393317.matw_international_google_ads.ads_AccountBasicStats_7713128947`
"""

REDSHIFT_SCHEMA = "bronze"
REDSHIFT_TABLE = "stg_ga_matw_international_google_ads"


# ===============================
# 🔹 Fetch data from BigQuery
# ===============================
client = bigquery.Client(project=BQ_PROJECT_ID)
query_job = client.query(BQ_QUERY)
df = query_job.to_dataframe()

print(f"✅ Pulled {len(df)} rows from BigQuery")


# ===============================
# 🔹 Function: Insert DataFrame into Redshift (via SSH Tunnel)
# ===============================
def insert_dataframe_to_redshift(df: pd.DataFrame, schema: str, table: str):
    """Insert DataFrame into Redshift using SSH tunnel + psycopg2 batch insert"""

    # Convert dict & list columns into strings
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, dict)).any():
            df[column] = df[column].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

        if df[column].apply(lambda x: isinstance(x, list)).any():
            df[column] = df[column].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

    # Replace NaN/None with None
    df = df.where(pd.notnull(df), None)

    with SSHTunnelForwarder(
        (SSH_CONFIG["ssh_host"], SSH_CONFIG["ssh_port"]),
        ssh_username=SSH_CONFIG["ssh_user"],
        ssh_password=SSH_CONFIG["ssh_password"],
        remote_bind_address=(REDSHIFT_CONFIG["host"], REDSHIFT_CONFIG["port"]),
        local_bind_address=("localhost", 5439)
    ) as tunnel:
        print("✅ SSH tunnel established")

        # Redshift connection through tunnel
        conn = psycopg2.connect(
            dbname=REDSHIFT_CONFIG["database"],
            user=REDSHIFT_CONFIG["user"],
            password=REDSHIFT_CONFIG["password"],
            host="localhost",
            port=tunnel.local_bind_port
        )
        cursor = conn.cursor()

        # Prepare INSERT SQL
        columns = ', '.join(f'"{col}"' for col in df.columns)
        insert_sql = f'INSERT INTO "{schema}"."{table}" ({columns}) VALUES %s'

        # Convert DataFrame to list of tuples
        rows = list(df.itertuples(index=False, name=None))

        # Batch insert
        batch_size = 5000
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i + batch_size]
            execute_values(cursor, insert_sql, batch)
            conn.commit()
            print(f"Inserted {min(i + batch_size, len(rows))}/{len(rows)} rows")

        cursor.close()
        conn.close()
        print(f"✅ Finished inserting {len(rows)} rows into {schema}.{table}")


# ===============================
# 🔹 Run Insert
# ===============================
insert_dataframe_to_redshift(df, REDSHIFT_SCHEMA, REDSHIFT_TABLE)
